In [ ]:
from datacube.utils.dask import start_local_dask
from dep_tools.namers import LocalPath
from dep_tools.writers import LocalDsCogWriter

from src.run_task import CoastalCompositesProcessor, get_geometry

from dep_tools.searchers import PystacSearcher
from dep_tools.loaders import OdcLoader

from coastlines.grids import VIETNAM_10

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Optionally set up a local dask cluster
client = start_local_dask()
client.dashboard_link

In [ ]:
import geopandas as gpd

# Study site configuration
tile_id = (9, 19)  # South west man face place
year = 2022  # Each year builds a composite 3 years

# Get the tile
geom = get_geometry(tile_id)
geom.explore()

In [ ]:
# Find some items
searcher = PystacSearcher(
    catalog="https://earth-search.aws.element84.com/v1/",
    collections=["sentinel-2-l2a"],
    datetime=f"{year-1}/{year+1}"
)

items = searcher.search(geom)

print(f"Found {len(items)} items")

In [ ]:
# Set up a data loader
loader = OdcLoader(
    crs=VIETNAM_10.crs,
    resolution=50,
    bands=["red", "green", "blue", "scl"],
    groupby="solar_day",
    chunks=dict(time=1, x=4096, y=4096),
    fail_on_error=False
)

# Run the load process, which is lazy-loaded
input_data = loader.load(items, geom)
input_data

In [ ]:
# Set up a data processor
processor = CoastalCompositesProcessor(
    low_or_high="high"
)

# Plan the processing.
output_data = processor.process(input_data)
output_data

In [ ]:
output = output_data.compute()
output

In [ ]:
output.odc.explore(vmin=0, vmax=2000)

In [ ]:
import xarray as xr

medians = []
for band in ["red", "green", "blue"]:
    medians.append(output[band].median("time"))

merged = xr.merge(medians)

In [ ]:
merged.odc.explore(vmin=0, vmax=2000)

In [ ]:
output_data.mean_vv.odc.write_cog("temp.tif")

In [ ]:
output_data["mean_vv_vh"] = (output_data["mean_vv"] / output_data["mean_vh"])
output_data[["mean_vv", "mean_vh", "mean_vv_vh"]].to_array().plot.imshow(size=10, vmin=0, vmax=0.1)

In [ ]:
# Testing the Azure writer

# from dep_tools.writers import AzureDsWriter
# from dep_tools.namers import DepItemPath

# itempath = DepItemPath("geomad", "test", "0.0", datetime)

# writer = AzureDsWriter(
#     itempath=itempath,
#     overwrite=True,
#     convert_to_int16=False,
#     extra_attrs=dict(dep_version="0.0"),
# )

# writer.write(output_data, "test")


In [ ]:
# Testing the AWS writer

from dep_tools.writers import AwsDsCogWriter
from dep_tools.namers import DepItemPath

itempath = DepItemPath("geomad", "test", "0.0", datetime)

writer = AwsDsCogWriter(
    itempath=itempath,
    overwrite=False,
    convert_to_int16=False,
    extra_attrs=dict(dep_version="0.0"),
    bucket="files.auspatious.com"
)

writer.write(output_data, "test")

In [ ]:
from odc.stac import load
from pystac import Item

item = Item.from_file("https://deppcpublicstorage.blob.core.windows.net/output/dep_geomad_test/0-0/test/2023-01/dep_geomad_test_test_2023-01.stac-item.json")

data = load([item], chunks={})
data

In [ ]:
# This is the target path
dep_path = LocalPath(
    local_folder="data",
    sensor="s1",
    dataset_id="mosaic",
    version="0.0.0",
    time=datetime,
    zero_pad_numbers=True
)

# Set up a writer and write out the files
writer = LocalDsWriter(
    itempath=dep_path,
    output_nodata=0,
    use_odc_writer=True,
    overwrite=True,
    convert_to_int16=False
)
print(f"Writing to: {dep_path._folder(item_id)}")
out_files = writer.write(output_data, item_id)

In [ ]:
# Make sure written files are readable
stac_path = writer.itempath.path(item_id, ext=".stac-item.json")

item = Item.from_file(stac_path)
item.validate()

In [ ]:
from pystac import Item
item = Item.from_file("https://s3.ap-southeast-2.amazonaws.com/files.auspatious.com/ausp_s2_low_tide_comp_20p/0-0-0/009/019/2022/ausp_s2_low_tide_comp_20p_009_019_2022.stac-item.json")

In [ ]:
from odc.stac import load
from pystac import Item

item = Item.from_file("https://s3.ap-southeast-2.amazonaws.com/files.auspatious.com/ausp_s2_low_tide_comp_20p/0-0-0/009/019/2022/ausp_s2_low_tide_comp_20p_009_019_2022.stac-item.json")

data = load([item], chunks={})

data.odc.explore(bands=["red", "green", "blue"], vmin=0, vmax=2000)